In [3]:
import re

def simplify_numbers(raw_text):
    def round_large_numbers(match):
        """Rounds large numbers based on rules."""
        num_str = match.group(0).replace(".", "").replace(",", ".")
        try:
            num = float(num_str)

            if num >= 1000000:
                rounded_num = int(round(num / 1000)) * 1000
            elif num >= 1000:
                rounded_num = int(round(num / 1000)) * 1000
            elif num >= 100:
                rounded_num = round(num / 10) * 10
            else:
                rounded_num = round(num)

            if rounded_num != num:
                return f"etwa {rounded_num:,}".replace(",", ".")
        except ValueError:
            pass
        return match.group(0)

    def simplify_percentages(match):
        """Simplifies percentages and returns common phrases."""
        percent = float(re.sub(r'[^\d.]', '', match.group(0).replace(",", ".")))

        if percent == 0:
            return "Keine"
        elif percent == 25:
            return "jeder Vierte (ein Viertel, also ein Stück von vier gleich großen Teilen)"
        elif percent == 50:
            return "die Hälfte (ein Teil von zwei gleich großen Teilen)"
        elif percent == 75:
            return "drei von vier (drei Viertel eines Ganzen)"
        elif percent == 100:
            return "Alle (das Ganze, ohne Ausnahmen)"
        elif percent < 25:
            return f"wenige ({int(round(percent))}% - kleiner als ein Viertel)"
        elif percent < 50:
            return f"weniger als die Hälfte ({int(round(percent))}% - kleiner als die Hälfte)"
        elif percent < 75:
            return f"mehr als die Hälfte ({int(round(percent))}% - größer als die Hälfte)"
        elif percent >= 90:
            return f"fast alle ({int(round(percent))}% - beinahe das Ganze)"
        return f"{int(round(percent))}%"

    def add_contextual_explanations(text):
        """Adds contextual explanations immediately after values in parentheses."""
        explanations = [
            (r'\b(1)\s?Million\b', "So viel Geld, dass man 100 Autos kaufen könnte"),
            (r'\b(10\.000|10000)\b\s+Besucher\b', "So viele Menschen, wie in ein großes Fußballstadion passen"),
            (r'(\b\d{1,3}(\.\d{3})*)\s+Menschen\b', "eine große Gruppe"),
            (r'\b(\d{2,3})\s*kilogramm\b', "so schwer wie ein großer Kühlschrank"),
            (r'\b(\d+,\d+|\d+)\s*Grad Celsius\b', "eine hohe Temperatur"),
            (r'\b(\d{1,3})\s*Prozent der Fläche\b', lambda m: "weniger als die Hälfte (30% - kleiner als die Hälfte)" if int(m.group(1)) == 30 else "ein Viertel"),
            (r'\b(\d{3,4})\s*(Jahre?|Jahr)\s*alt', lambda m: f"{m.group(0)} (zwischen {int(m.group(1)) - 100} und {int(m.group(1)) + 100} Jahre alt, vergleichbar mit der Zeit der alten Römer)"),
            (r'(\b\d{1,3}(\.\d{3})*)\s*(Millionen?|Euro|USD|Dollar|Pfund|Yen|Rupien|Franken)\b', lambda m: f"{m.group(0)} (eine große Summe, oft vergleichbar mit dem Preis eines Hauses oder mehr)"),
            (r'(\b\d{1,3}(\.\d{3})*)\s*(Ereignisse|Vorfälle)\b', lambda m: f"{m.group(0)} (so viele Ereignisse wie an einem belebten Tag in einer Großstadt, wo ständig etwas passiert)"),
        ]
        
        # Regular expression to match ranges for Ereignisse
        # Matching numbers followed by "Ereignisse"
        text = re.sub(r'(\b\d{1,3}(\.\d{3})*)\s*(Ereignisse|Vorfälle)\s+und\s+(\b\d{1,3}(\.\d{3})*)\s*(Ereignisse|Vorfälle)', lambda m: f"{m.group(0)} (von {m.group(1)} bis {m.group(4)} Ereignissen – das ist eine beachtliche Spanne, vergleichbar mit dem Unterschied zwischen einem normalen und einem außergewöhnlich hektischen Tag)", text)
        
        for pattern, explanation in explanations:
            text = re.sub(pattern, lambda m: f"{m.group(0)} ({explanation if isinstance(explanation, str) else explanation(m)})", text)
        
        return text

    def simplify_numbers_in_text(text):
        """Main function to process the text."""
        preserved = {}
        def preserve_match(match, format):
            key = f"@@PRESERVED_{format}_{len(preserved)}@@"
            preserved[key] = match.group(0)
            return key

        # Patterns to preserve various date formats
        date_patterns = [
            (r'\b(?:Im Jahr|im Jahr|Jahre?|im Jahre|Jahre | Jahr | jahre | jahre)\s\d{1,4}\b', 'year'),
            (r'\bAm\s(?:0?[1-9]|[12][0-9]|3[01])\s(?:Januar|Februar|März|April|Mai|Juni|Juli|August|September|Oktober|November|Dezember)\s\d{4}\b', 'full_date'),
            (r'\b\d{1,2}\.\s(?:Januar|Februar|März|April|Mai|Juni|Juli|August|September|Oktober|November|Dezember)\s\d{4}\b', 'full_date'),
            (r'\b\d{1,2}\./\d{1,2}\./\d{4}\b', 'iso_date'),
            (r'\b(?:Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),\s(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}\b', 'day_month_year')
        ]

        for pattern, format_type in date_patterns:
            text = re.sub(pattern, lambda match: preserve_match(match, format_type), text)

        # Simplify numbers and percentages
        text = re.sub(r'\b\d{1,3}(\.\d{3})*(,\d{1,2})?\b', round_large_numbers, text)
        text = re.sub(r'\b\d+[,\.]?\d*\b', round_large_numbers, text)
        text = re.sub(r'\b\d+\s*[\.,]?\d*\s*(?:%|Prozent|prozent|Percent|percent)\b', simplify_percentages, text)

        # Restore preserved date formats
        for key, value in preserved.items():
            text = text.replace(key, value)

        # Add contextual explanations for various units
        text = add_contextual_explanations(text)

        return text

    return simplify_numbers_in_text(raw_text)

def main():
    test_cases = [
      "324.620,22 Euro wurden gespendet.",
      "1.897 Menschen nahmen teil.",
      "25 Prozent der Bevölkerung sind betroffen.",
      "90 Prozent stimmten zu.",
      "14 Prozent lehnten ab.",
      "Bei 38,7 Grad Celsius ist es sehr heiß.",
      "denn die Rente steigt um 4,57 Prozent.",
      "Im Jahr 2024 gab es 1.234 Ereignisse.",
      "Am 1. Januar 2024 waren es 5.678 Teilnehmer.",
      "Im Jahr 2025 gab es 2.018 Ereignisse."
    ]

    for case in test_cases:
        print(simplify_numbers(case))

if __name__ == "__main__":
    main()


etwa 325.000 Euro (325.000 Euro (eine große Summe, oft vergleichbar mit dem Preis eines Hauses oder mehr)) wurden gespendet.
etwa 2.000 Menschen (eine große Gruppe) nahmen teil.
jeder Vierte (ein Viertel, also ein Stück von vier gleich großen Teilen) der Bevölkerung sind betroffen.
fast alle (90% - beinahe das Ganze) stimmten zu.
wenige (14% - kleiner als ein Viertel) lehnten ab.
Bei etwa 39 Grad Celsius (eine hohe Temperatur) ist es sehr heiß.
denn die Rente steigt um etwa wenige (5% - kleiner als ein Viertel).
Im Jahr 2024 gab es etwa 1.000 Ereignisse (1.000 Ereignisse (so viele Ereignisse wie an einem belebten Tag in einer Großstadt, wo ständig etwas passiert)).
Am 1. Januar 2024 waren es etwa 6.000 Teilnehmer.
Im Jahr 2025 gab es etwa 2.000 Ereignisse (2.000 Ereignisse (so viele Ereignisse wie an einem belebten Tag in einer Großstadt, wo ständig etwas passiert)).
